<a href="https://colab.research.google.com/github/fatimainy262/pdc-gray-scale/blob/main/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# library for reading images
!wget https://raw.githubusercontent.com/nothings/stb/master/stb_image.h

# library for writing images
!wget https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h

--2025-12-09 18:31:14--  https://raw.githubusercontent.com/nothings/stb/master/stb_image.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 283010 (276K) [text/plain]
Saving to: ‘stb_image.h’

stb_image.h         100%[===================>] 276.38K  --.-KB/s    in 0.003s  

2025-12-09 18:31:15 (93.6 MB/s) - ‘stb_image.h’ saved [283010/283010]

--2025-12-09 18:31:15--  https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71221 (70K) [text/plain]
Saving to: ‘stb_image_

In [2]:
%%writefile parallel_filter_v2.cpp
#include "stb_image.h"
#include "stb_image_write.h"

// Required for OpenMP functions and directives
#include <omp.h>
#include <iostream>
#include <string>
#include <cstdlib> // Required for exit()

// Define the implementation of the image libraries *once* in the source file
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

// Function to handle the image processing for a single file
void process_image(const std::string& input_path, const std::string& output_path) {
    int width, height, channels;

    // 1. Read the image
    // Load the image with 3 channels (RGB)
    unsigned char *data = stbi_load(input_path.c_str(), &width, &height, &channels, 3);

    if (data == NULL) {
        std::cerr << "Error: Could not load image from path: " << input_path << std::endl;
        return;
    }

    std::cout << "Processing " << input_path << " (" << width << "x" << height << ") in parallel..." << std::endl;

    // Calculate total number of pixels
    int total_pixels = width * height;

    // 2. Parallel Grayscale Processing Loop
    // The #pragma omp parallel for distributes the loop iterations (pixels)
    // among multiple threads.
    #pragma omp parallel for
    for (int i = 0; i < total_pixels; ++i) {
        // Calculate the starting index of the R, G, B components for the current pixel
        int pixel_start_index = i * 3;

        // Read the R, G, B values
        int R = data[pixel_start_index];
        int G = data[pixel_start_index + 1];
        int B = data[pixel_start_index + 2];

        // Apply the Luminance formula to calculate the Gray Value (Y)
        unsigned char Y = (unsigned char)(0.299 * R + 0.587 * G + 0.114 * B);

        // Write the new value back to the array for all three channels (R=G=B=Y)
        data[pixel_start_index]     = Y; // R
        data[pixel_start_index + 1] = Y; // G
        data[pixel_start_index + 2] = Y; // B
    }

    // 3. Write the image
    // Using PNG for output
    int success = stbi_write_png(output_path.c_str(), width, height, 3, data, width * 3);

    if (success) {
        std::cout << "Success! Grayscale image saved to: " << output_path << std::endl;
    } else {
        std::cerr << "Error: Failed to save image to: " << output_path << std::endl;
    }

    // Free the image data from memory
    stbi_image_free(data);
}

// Main function to read arguments
// argc is the count of arguments, argv is the array of argument strings
int main(int argc, char* argv[]) {
    // Check if the user provided the correct number of arguments (3 total: program name, input file, output file)
    if (argc != 3) {
        std::cerr << "Usage: ./" << argv[0] << " <input_image_path> <output_image_path>" << std::endl;
        std::cerr << "Example: ./filter_exec my_color_photo.jpg my_grayscale_output.png" << std::endl;
        // Exit the program with an error code
        return 1;
    }

    // Get the input and output paths from the command line arguments
    std::string input_path = argv[1];
    std::string output_path = argv[2];

    // Process the image using the provided paths
    process_image(input_path, output_path);

    return 0;
}

Writing parallel_filter_v2.cpp
